# Original To Curated

The purpose of this notebook is to read the data issued from the [zenado website](https://zenodo.org/record/5530410) and stored in the `Dada/Original/` folder, to aggregate and clean the data, and to store it in the `Data/Curated/` folder in an appropriate format. 

In [1]:
import pandas as pd
import numpy as np

Some variables to point to the origin folders and the curated folders.

In [2]:
dataOriginalPath = 'Data/Clickbait17/Original/'
dataTrainX = dataOriginalPath +'clickbait17-train-170630/instances.jsonl'
dataTrainY = dataOriginalPath +'clickbait17-train-170630/truth.jsonl'

# These are are two other files
dataTrainX1 = dataOriginalPath +'clickbait17-train-170331/instances.jsonl'
dataTrainY1 = dataOriginalPath +'clickbait17-train-170331/truth.jsonl'

dataTestX = dataOriginalPath +'clickbait17-test-170720/instances.jsonl'
dataTestY = dataOriginalPath +'clickbait17-test-170720/truth.jsonl'

dataCuratedPath = 'Data/Clickbait17/Curated/'
actualOutput = 'truthMean'

Reading the training data, using the [json lines](https://jsonlines.org/) format.

In [3]:
# Do not use automatic type detection because id are sometime converted to int64 and are truncated (4 hours bug).
pdoTrainX=pd.read_json(dataTrainX, lines = True, dtype=False)

In [4]:
pdoTrainX

postMedia  \
0                                        []   
1                                        []   
2                                        []   
3                                        []   
4                                        []   
...                                     ...   
19533  [media/photo_804240867972304896.jpg]   
19534                                    []   
19535                                    []   
19536  [media/photo_804133521023324160.jpg]   
19537  [media/photo_804125377400553474.jpg]   

                                                postText                  id  \
0      [UK’s response to modern slavery leaving victi...  858462320779026433   
1                                         [this is good]  858421020331560960   
2      [The "forgotten" Trump roast: Relive his bruta...  858368123753435136   
3                 [Meet the happiest #dog in the world!]  858323428260139008   
4      [Tokyo's subway is shut down amid fears over a...  858283602626347008   
...                                                  ...                 ...   
19533  [Brazil soccer team and pilot's final intervie...  804250183642976256   
19534                                   [😱😱😱😱😱😱😱😱😱😱😱😱😱😱]  804156272086020096   
19535  [Frenchs Forest high school may have to make w...  804149798651588608   
19536                                   [Oh Jeff… #bruh]  804134698729385984   
19537  [Richard Sherman weighs in on Cam Newton’s str...  804126501117435904   

                                          targetCaptions  \
0                               [modern-slavery-rex.jpg]   
1      [In this July 1, 2010 file photo, Dr. Charmain...   
2      [President Trump will not attend this year's W...   
3                        [Maru , Maru, Maru, Maru, Maru]   
4      [All nine lines of Tokyo's subway system were ...   
...                                                  ...   
19533            [CNBC, msnbc, NBC NEWS, TODAY, xfinity]   
19534  [Instagram/madonna, Speaker Ryan Retreats on H...   
19535  [An artist's impression of the proposed new to...   
19536  [Jeff Fisher May Think Danny Woodhead Still Pl...   
19537  [Richard Sherman Says 'Karma' Caught Up to Cam...   

                                        targetParagraphs  \
0      [Thousands of modern slavery victims have not ...   
1      [President Donald Trump has appointed the pro-...   
2      [When the White House correspondents’ dinner i...   
3      [Adorable is probably an understatement. This ...   
4      [One of Tokyo's major subways systems says it ...   
...                                                  ...   
19533  [Watch Live: Joe Biden Honored on Senate Floor...   
19534  [On November 30 Politico reported that Eric Tr...   
19535  [The Forest High School on Sydney's northern b...   
19536  [NFL coaches have a lot of information to reme...   
19537  [Seattle Seahawks cornerback Richard Sherman m...   

                                             targetTitle  \
0      ‘Inexcusable’ failures in UK’s response to mod...   
1      Donald Trump Appoints Pro-Life Advocate as Ass...   
2      The ‘forgotten’ Trump roast: Relive his brutal...   
3      Meet The Happiest Dog In The World, Maru The H...   
4      Tokyo's subway is shut down amid fears over an...   
...                                                  ...   
19533  NBC News Video See Brazil Soccer Team, Pilot’s...   
19534         Politico Scoop: Eric Trump Killed Two Deer   
19535  Frenchs Forest high school may relocate to mak...   
19536  Los Angeles Rams Jeff Fisher May Think Danny W...   
19537  Seattle Seahawks Richard Sherman Says 'Karma' ...   

                        postTimestamp  \
0      Sat Apr 29 23:25:41 +0000 2017   
1      Sat Apr 29 20:41:34 +0000 2017   
2      Sat Apr 29 17:11:23 +0000 2017   
3      Sat Apr 29 14:13:46 +0000 2017   
4      Sat Apr 29 11:35:31 +0000 2017   
...                               ...   
19533  Thu Dec 01 09:06:00 +0000 2016   
19534  Thu Dec 01

Dropping some unused columns but keeping some of them for debugging purposes.

In [5]:
# Just object types
pdoTrainX.dtypes

postMedia            object
postText             object
id                   object
targetCaptions       object
targetParagraphs     object
targetTitle          object
postTimestamp        object
targetKeywords       object
targetDescription    object
dtype: object

In [6]:
droppedColumns = ['targetCaptions', 'targetDescription', 'targetParagraphs', 'postTimestamp' , 'targetKeywords']

In [7]:
pdoTrainX.drop(droppedColumns, axis=1, inplace=True)

In [8]:
# Use id as an index. Dont forget inplace=True
pdoTrainX.set_index('id', inplace=True)

In [9]:
pdoTrainX

postMedia  \
id                                                         
858462320779026433                                    []   
858421020331560960                                    []   
858368123753435136                                    []   
858323428260139008                                    []   
858283602626347008                                    []   
...                                                  ...   
804250183642976256  [media/photo_804240867972304896.jpg]   
804156272086020096                                    []   
804149798651588608                                    []   
804134698729385984  [media/photo_804133521023324160.jpg]   
804126501117435904  [media/photo_804125377400553474.jpg]   

                                                             postText  \
id                                                                      
858462320779026433  [UK’s response to modern slavery leaving victi...   
858421020331560960                                     [this is good]   
858368123753435136  [The "forgotten" Trump roast: Relive his bruta...   
858323428260139008             [Meet the happiest #dog in the world!]   
858283602626347008  [Tokyo's subway is shut down amid fears over a...   
...                                                               ...   
804250183642976256  [Brazil soccer team and pilot's final intervie...   
804156272086020096                                   [😱😱😱😱😱😱😱😱😱😱😱😱😱😱]   
804149798651588608  [Frenchs Forest high school may have to make w...   
804134698729385984                                   [Oh Jeff… #bruh]   
804126501117435904  [Richard Sherman weighs in on Cam Newton’s str...   

                                                          targetTitle  
id                                                                     
858462320779026433  ‘Inexcusable’ failures in UK’s response to mod...  
858421020331560960  Donald Trump Appoints Pro-Life Advocate as Ass...  
858368123753435136  The ‘forgotten’ Trump roast: Relive his brutal...  
858323428260139008  Meet The Happiest Dog In The World, Maru The H...  
858283602626347008  Tokyo's subway is shut down amid fears over an...  
...                                                               ...  
804250183642976256  NBC News Video See Brazil Soccer Team, Pilot’s...  
804156272086020096         Politico Scoop: Eric Trump Killed Two Deer  
804149798651588608  Frenchs Forest high school may relocate to mak...  
804134698729385984  Los Angeles Rams Jeff Fisher May Think Danny W...  
804126501117435904  Seattle Seahawks Richard Sherman Says 'Karma' ...  

[19538 rows x 3 columns]

In [10]:
# Remove the brackets from postText column, meaning taking the first element of the one element list
pdoTrainX["postText"]=pdoTrainX["postText"].apply(lambda x : x[0])
pdoTrainX["postMedia"]=pdoTrainX["postMedia"].apply(lambda x : x[0] if len(x)>0 else '')


In [11]:
# C:Chalenge dataset, B:Big dataset(Chalenge + Supplement), T:Test dataset, S:Supplement
pdoTrainX["fromDataset"] = "C"

Reading the actual output values

In [12]:
pdoTrainY=pd.read_json(dataTrainY, lines = True, dtype=False)

In [13]:
pdoTrainY

truthJudgments  truthMean  \
0                              [1.0, 1.0, 1.0, 1.0, 1.0]   1.000000   
1      [0.33333333330000003, 0.0, 0.33333333330000003...   0.133333   
2      [0.33333333330000003, 0.6666666666000001, 1.0,...   0.400000   
3      [0.0, 0.6666666666000001, 0.0, 0.3333333333000...   0.266667   
4                              [0.0, 0.0, 0.0, 0.0, 0.0]   0.000000   
...                                                  ...        ...   
19533           [0.0, 0.6666666666000001, 0.0, 0.0, 0.0]   0.133333   
19534          [0.0, 0.0, 0.0, 0.33333333330000003, 0.0]   0.066667   
19535  [0.6666666666000001, 0.6666666666000001, 0.0, ...   0.333333   
19536           [1.0, 0.0, 0.6666666666000001, 1.0, 1.0]   0.733333   
19537          [0.0, 0.0, 0.0, 0.33333333330000003, 0.0]   0.066667   

                       id    truthClass  truthMedian  truthMode  
0      858464162594172928     clickbait     1.000000   1.000000  
1      858462320779026433  no-clickbait     0.000000   0.000000  
2      858460992073863168  no-clickbait     0.333333   0.000000  
3      858459539296980995  no-clickbait     0.333333   0.333333  
4      858455355948384257  no-clickbait     0.000000   0.000000  
...                   ...           ...          ...        ...  
19533  804126501117435904  no-clickbait     0.000000   0.000000  
19534  804123103995580416  no-clickbait     0.000000   0.000000  
19535  804121272967983104  no-clickbait     0.333333   0.000000  
19536  804119512010424320     clickbait     1.000000   1.000000  
19537  804113781580328960  no-clickbait     0.000000   0.000000  

[19538 rows x 6 columns]

In [14]:
pdoTrainY.set_index('id', inplace=True)

In [15]:
pdoTrainY

truthJudgments  \
id                                                                      
858464162594172928                          [1.0, 1.0, 1.0, 1.0, 1.0]   
858462320779026433  [0.33333333330000003, 0.0, 0.33333333330000003...   
858460992073863168  [0.33333333330000003, 0.6666666666000001, 1.0,...   
858459539296980995  [0.0, 0.6666666666000001, 0.0, 0.3333333333000...   
858455355948384257                          [0.0, 0.0, 0.0, 0.0, 0.0]   
...                                                               ...   
804126501117435904           [0.0, 0.6666666666000001, 0.0, 0.0, 0.0]   
804123103995580416          [0.0, 0.0, 0.0, 0.33333333330000003, 0.0]   
804121272967983104  [0.6666666666000001, 0.6666666666000001, 0.0, ...   
804119512010424320           [1.0, 0.0, 0.6666666666000001, 1.0, 1.0]   
804113781580328960          [0.0, 0.0, 0.0, 0.33333333330000003, 0.0]   

                    truthMean    truthClass  truthMedian  truthMode  
id                                                                   
858464162594172928   1.000000     clickbait     1.000000   1.000000  
858462320779026433   0.133333  no-clickbait     0.000000   0.000000  
858460992073863168   0.400000  no-clickbait     0.333333   0.000000  
858459539296980995   0.266667  no-clickbait     0.333333   0.333333  
858455355948384257   0.000000  no-clickbait     0.000000   0.000000  
...                       ...           ...          ...        ...  
804126501117435904   0.133333  no-clickbait     0.000000   0.000000  
804123103995580416   0.066667  no-clickbait     0.000000   0.000000  
804121272967983104   0.333333  no-clickbait     0.333333   0.000000  
804119512010424320   0.733333     clickbait     1.000000   1.000000  
804113781580328960   0.066667  no-clickbait     0.000000   0.000000  

[19538 rows x 5 columns]

Combining input values and output values

In [16]:
# Not necessary to do that. Should return True anyway
pdoTrainX.sort_index(inplace=True)
pdoTrainY.sort_index(inplace=True)
pdoTrainX.index.equals(pdoTrainY.index)

True

In [17]:
# This use the id index to affect values at the proper place
pdoTrainX['truthMean']=pdoTrainY['truthMean']

In [18]:
pdoTrainX

postMedia  \
id                                                       
804113781580328960                                       
804119512010424320  media/photo_804119509338640385.jpg   
804121272967983104                                       
804123103995580416                                       
804126501117435904  media/photo_804125377400553474.jpg   
...                                                ...   
858455355948384257                                       
858459539296980995  media/photo_858459536574828544.jpg   
858460992073863168  media/photo_858460986612862976.jpg   
858462320779026433                                       
858464162594172928  media/photo_858026213083799552.jpg   

                                                             postText  \
id                                                                      
804113781580328960  Panama Papers: Europol links 3,500 names to su...   
804119512010424320                The key to truly great chicken soup   
804121272967983104  Afghan policewomen face down their fears to serve   
804123103995580416  Conservatives are watching less football this ...   
804126501117435904  Richard Sherman weighs in on Cam Newton’s stru...   
...                                                               ...   
858455355948384257  Trump now agrees with the majority of American...   
858459539296980995  Trump has flip-flopped. But his supporters are...   
858460992073863168                Inside North Korea's secret prisons   
858462320779026433  UK’s response to modern slavery leaving victim...   
858464162594172928   Facts that will be truly upsetting to '90s girls   

                                                          targetTitle  \
id                                                                      
804113781580328960  Panama Papers: Europol links 3,500 names to su...   
804119512010424320                           A Superior Chicken Soup    
804121272967983104  100 Women 2016: On the frontline with the wome...   
804123103995580416  Older Viewers and Conservatives Are Watching L...   
804126501117435904  Seattle Seahawks Richard Sherman Says 'Karma' ...   
...                                                               ...   
858455355948384257  Donald Trump said being US president was harde...   
858459539296980995  Trump Has Flip-Flopped. But His Supporters Are...   
858460992073863168  Inside Kim Jong-un's camps of death: Former No...   
858462320779026433  ‘Inexcusable’ failures in UK’s response to mod...   
858464162594172928   Facts That Will Be Truly Upsetting To '90s Girls   

                   fromDataset  truthMean  
id                                         
804113781580328960           C   0.066667  
804119512010424320           C   0.733333  
804121272967983104           C   0.333333  
804123103995580416           C   0.066667  
804126501117435904           C   0.133333  
...                        ...        ...  
858455355948384257           C   0.000000  
858459539296980995           C   0.266667  
858460992073863168           C   0.400000  
858462320779026433           C   0.133333  
858464162594172928           C   1.000000  

[19538 rows x 5 columns]

In [19]:
pdoTrainX.describe()

truthMean
count  19538.000000
mean       0.324530
std        0.252824
min        0.000000
25%        0.133333
50%        0.266667
75%        0.466667
max        1.000000

Same thing for supplements values

In [20]:
pdoTrainX1=pd.read_json(dataTrainX1, lines = True, dtype=False)
pdoTrainX1.drop(droppedColumns, axis=1, inplace=True)
pdoTrainX1.set_index('id', inplace=True)
pdoTrainX1["postText"]=pdoTrainX1["postText"].apply(lambda x : x[0])
pdoTrainX1["postMedia"]=pdoTrainX1["postMedia"].apply(lambda x : x[0] if len(x)>0 else '')

pdoTrainY1=pd.read_json(dataTrainY1, lines = True, dtype=False)
pdoTrainY1.set_index('id', inplace=True)

pdoTrainX1.sort_index(inplace=True)
pdoTrainY1.sort_index(inplace=True)
pdoTrainX1['truthMean']=pdoTrainY1['truthMean']

pdoTrainX1["fromDataset"] = "S" # C:Chalenge dataset, B:Big dataset(Chalenge + Supplement), T:Test dataset, S:Supplement

Same thing for test values

In [26]:
pdoTestX=pd.read_json(dataTestX, lines = True, dtype=False)
pdoTestX.drop(droppedColumns, axis=1, inplace=True)
pdoTestX.set_index('id', inplace=True)
pdoTestX["postText"]=pdoTestX["postText"].apply(lambda x : x[0])
pdoTestX["postMedia"]=pdoTestX["postMedia"].apply(lambda x : x[0] if len(x)>0 else '')

pdoTestY=pd.read_json(dataTestY, lines = True, dtype=False)
pdoTestY.set_index('id', inplace=True)

pdoTestX.sort_index(inplace=True)
pdoTestY.sort_index(inplace=True)
pdoTestX['truthMean']=pdoTestY['truthMean']

pdoTestX["fromDataset"] = "T" # C:Chalenge dataset, B:Big dataset(Chalenge + Supplement), T:Test dataset, S:Supplement

In [28]:
pd.concat([pdoTrainX, pdoTrainX1, pdoTestX])

postMedia  \
id                                                       
804113781580328960                                       
804119512010424320  media/photo_804119509338640385.jpg   
804121272967983104                                       
804123103995580416                                       
804126501117435904  media/photo_804125377400553474.jpg   
...                                                ...   
858460992052834304  media/photo_858460989590843398.jpg   
858461421646086145                                       
858462092407562244  media/photo_858462089165258752.jpg   
858462894568165376                                       
858465168795332610                                       

                                                             postText  \
id                                                                      
804113781580328960  Panama Papers: Europol links 3,500 names to su...   
804119512010424320                The key to truly great chicken soup   
804121272967983104  Afghan policewomen face down their fears to serve   
804123103995580416  Conservatives are watching less football this ...   
804126501117435904  Richard Sherman weighs in on Cam Newton’s stru...   
...                                                               ...   
858460992052834304  In President Trump's absence, "nerd prom" is c...   
858461421646086145  Here's a breakdown of the best and riskiest mo...   
858462092407562244  Austin, Texas police officer faked his death a...   
858462894568165376                     Motherhood in the time of Zika   
858465168795332610                             that escalated quickly   

                                                          targetTitle  \
id                                                                      
804113781580328960  Panama Papers: Europol links 3,500 names to su...   
804119512010424320                           A Superior Chicken Soup    
804121272967983104  100 Women 2016: On the frontline with the wome...   
804123103995580416  Older Viewers and Conservatives Are Watching L...   
804126501117435904  Seattle Seahawks Richard Sherman Says 'Karma' ...   
...                                                               ...   
858460992052834304  Plan Samantha Bee: Late Night Comedian Hosts A...   
858461421646086145  Best and riskiest moves for every team's 2017 ...   
858462092407562244  Texas police officer faked his death and fled ...   
858462894568165376                     Motherhood in the time of Zika   
858465168795332610  Maher: Trump’s Tax Plan Proves ‘Mental Illness...   

                   fromDataset  truthMean  
id                                         
804113781580328960           C   0.066667  
804119512010424320           C   0.733333  
804121272967983104           C   0.333333  
804123103995580416           C   0.066667  
804126501117435904           C   0.133333  
...                        ...        ...  
858460992052834304           T   0.266667  
858461421646086145           T   0.600000  
858462092407562244           T   0.066667  
858462894568165376           T   0.333333  
858465168795332610           T   1.000000  

[40976 rows x 5 columns]

In [29]:
# Convert the result into a Huggingface dataset
import datasets as ds
curated = ds.Dataset.from_pandas(pd.concat([pdoTrainX, pdoTrainX1, pdoTestX])) #, preserve_index=False)

In [30]:
curated.info.description = "Clickbait"
curated.info.version = "0.6.0"
curated.info.supervised_keys = [actualOutput]
curated.save_to_disk(dataCuratedPath)

In [31]:
# This file is not used. The \n\n line characters are used in the postText
curated.to_csv(dataCuratedPath+"/dataset.csv", sep=';') 

Creating CSV from Arrow format: 100%|██████████| 5/5 [00:00<00:00, 11.34ba/s]


8606387

In [32]:
# This file is not used. The \n\n line characters are used in the postText
curated.to_json(dataCuratedPath+"/dataset.jsonl") 

Creating json from Arrow format: 100%|██████████| 5/5 [00:00<00:00, 21.47ba/s]


11564270